## Случайный лес   ( Random Forest )

In [1]:
#  Подключаю необходимые библиотеки

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline


from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics




###  Подготовка данных:  распознавание качества вина

In [2]:
#  Задаю рабочую папку

import os
os.chdir("C:/Users/USER/Documents/Python/_AD_Python_part_1_Lectures\AD_Python_1_14_wine quality")


In [3]:
#  Импортирую данные

df = pd.read_csv('winequality-white.csv', sep=';')


In [4]:
# Проверяю себя
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
# Проверяю себя
#  Проверяю размер матрицы данных
df.shape

(4898, 12)

In [6]:
# Отбрасываю строки с пропусками. Слишком грубый подход, часто можно действовать лучше.
# df = df.dropna()
# df.shape

In [7]:
# Считаю, сколько пропусков в каждом столбце.
# Если пропуски найдутся, данные надо будет лечить
NAs = pd.concat([df.isnull().sum()], axis=1)
NAs[NAs.sum(axis=1) > 0]

,0


In [8]:
NAs

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


In [9]:
df.describe(include='all')

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [10]:
#  Классы сбалансированы?

df['quality'].value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [11]:
# Перекодирую редкие класы
df['quality'] = df['quality'].replace(to_replace=3, value=4).replace(9, 8)


In [12]:
# Проверяю себя
df['quality'].value_counts()

6    2198
5    1457
7     880
4     183
8     180
Name: quality, dtype: int64

In [13]:
#  разделяю предикоторы и отклики
y = df['quality']
X = df.drop(columns=['quality'])

In [16]:
#  Стандартизация предикторов нужна?

In [17]:
#  Проверяю себя
#  X.head()

#   Проверяю себя
#   y.head()

In [14]:
#  Разделяю на обучающую и тестовую выборку

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1234)  

##   Random Forest
##   Ручной подбор внешних параметров

In [20]:

model = RandomForestClassifier(random_state=42,
                               # число деревьев в лесу
                               n_estimators=80,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # то ли число слоев, то ли число расщеплений  
                               #  ?????????????????  но не число узлов
                               max_depth=5,
                               # сколько переменных использовать при построении дерева?
                               # sqrt(k)
                               max_features='auto', 
                               # максимальное число конечных узлов. Сейчас - без ограничений
                               max_leaf_nodes=None,
                               # минимальное уменьшение загрязнения 
                               min_impurity_decrease=0.001, 
                               # Устарело
                               min_impurity_split=None,
                               # Дерево обучается по части наблюдений (По какой???)
                               bootstrap=True, 
                               # минимальное число наблюдений в узле потомке
                               min_samples_leaf=10, 
                               # минимальное число наблюдений в узле родителе
                               min_samples_split=15,
                               # параметр используется редко, самостоятельно
                               min_weight_fraction_leaf=0.0, 
                               # объем промежуточных результатов в ходе обучения
                               verbose=1, 
                               # число используемых процессоров
                               # n_jobs=None,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения
                               class_weight=None)

model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.2s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.001, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, n_estimators=80,
                       n_jobs=None, oob_score=True, random_state=42, verbose=1,
                       warm_start=False)

In [21]:
# Распознаю классы с помощью модели
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.0s finished


In [24]:
#  Считаю accuracy модели

from sklearn.metrics import accuracy_score

print("Accuracy на обучающем множестве ", accuracy_score(y_train, y_pred_train)*100)
print("Accuracy на тестовом множестве ", accuracy_score(y_test, y_pred_test)*100)

Accuracy на обучающем множестве  58.03108808290155
Accuracy на тестовом множестве  55.473098330241186


In [26]:
#  Считаю F1 модели

print(metrics.classification_report(y_test,  y_pred_test))

              precision    recall  f1-score   support

           4       0.00      0.00      0.00        60
           5       0.60      0.58      0.59       469
           6       0.53      0.78      0.63       742
           7       0.56      0.17      0.26       283
           8       0.00      0.00      0.00        63

    accuracy                           0.55      1617
   macro avg       0.34      0.31      0.30      1617
weighted avg       0.52      0.55      0.51      1617



C:\Users\USER\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
#   Информативность переменных

pd.DataFrame({'feature': X_train.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

,feature,importance
10,alcohol,0.294905
1,volatile acidity,0.171276
7,density,0.150640
4,chlorides,0.073396
5,free sulfur dioxide,0.064665
3,residual sugar,0.055579
6,total sulfur dioxide,0.053697
2,citric acid,0.043449
8,pH,0.038728
9,sulphates,0.027170


In [27]:
#   print('Out-of-bag score: {0}'.format(model.oob_score_))

In [29]:
#   Таблица сопряженности.  Тестовое множество
#   Некрасивая таблица:  нет кодов классов!
metrics.confusion_matrix(y_test, y_pred_test)

array([[  0,  27,  33,   0,   0],
       [  0, 272, 196,   1,   0],
       [  0, 141, 576,  25,   0],
       [  0,  12, 222,  49,   0],
       [  0,   0,  50,  13,   0]], dtype=int64)

In [31]:
#   Таблица сопряженности.  Тестовое множество
#   Так красивее

conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,4,5,6,7,8
4,0,27,33,0,0
5,0,272,196,1,0
6,0,141,576,25,0
7,0,12,222,49,0
8,0,0,50,13,0


In [34]:
#   Таблица сопряженности.  Обучающее множество

conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,4,5,6,7,8
4,1,71,50,1,0
5,0,615,370,3,0
6,0,237,1183,36,0
7,0,17,427,153,0
8,0,0,86,31,0


Видим, что есть ошибки. <br>
Классы "4" и "8" не распознаются. <br>
Возможно, придется строить отдельные бинарные классификаторы для этих классов

In [44]:
#  С помощью модели оцениваем вероятности принадлежать классам 

y_pred_train2 = model.predict_proba(X_train)
y_pred_test2 = model.predict_proba(X_test)


In [45]:
print(y_pred_train2)
print(y_pred_test2)

[[1.53684706e-03 1.16732053e-02 8.11103044e-02 ... 3.96480060e-01
  9.04695810e-02 2.80781367e-03]
 [2.95275541e-03 1.90899053e-02 2.17209529e-01 ... 1.93216878e-01
  3.76044566e-02 1.66925107e-03]
 [2.48476070e-03 2.89730662e-02 5.11042514e-01 ... 6.17760227e-02
  1.41570434e-02 2.90876339e-04]
 ...
 [1.21543501e-03 1.77896984e-01 9.99599793e-02 ... 2.49391774e-01
  6.24365822e-02 3.07132672e-03]
 [3.76874880e-03 1.48365811e-02 1.79115253e-01 ... 1.64869809e-01
  3.98908617e-02 2.49702926e-04]
 [2.21945996e-03 1.81819058e-02 2.47444672e-01 ... 1.51046208e-01
  3.28948693e-02 1.95139742e-03]]
[[2.37194285e-03 1.50111951e-02 1.65074358e-01 ... 2.45659948e-01
  5.59871420e-02 9.53772623e-04]
 [1.49987232e-03 2.19333607e-02 3.67307161e-01 ... 1.24390430e-01
  6.50081383e-02 4.70151939e-04]
 [1.83068713e-03 2.49161101e-02 2.44996706e-01 ... 1.49329563e-01
  3.12846681e-02 2.44528415e-04]
 ...
 [3.87717430e-03 1.91756339e-02 2.48302095e-01 ... 1.44058664e-01
  3.10828374e-02 1.63685739e-03]

##   Random Forest
##   Подбор внешних параметров с помощью GridSearch

In [18]:
from sklearn.model_selection import GridSearchCV


In [19]:
#  Так будем делать на практике.
#  Но для лекции в решетке слишком много вариантов

#  param_grid = {  
#      'n_estimators': [30, 40, 50, 100],
#      'max_depth': [4, 5, 6],
#      "max_features": [1, 3, 10],
#      "min_samples_split": [3, 5, 10],
#  }

In [20]:
#  Экономный сокращенный вариант решетки

param_grid = {  
    'n_estimators': [30, 100, 200],
    'max_depth': [4, 5, 6],
}

In [22]:
model2 = RandomForestClassifier(random_state=42,
                               # число деревьев в лесу
                               n_estimators=80,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # то ли число слоев, то ли число расщеплений  
                               #  ?????????????????  но не число узлов
                               max_depth=5,
                               # сколько переменных использовать при построении дерева?
                               # sqrt(k)
                               max_features='auto', 
                               # максимальное число конечных узлов. Сейчас - без ограничений
                               max_leaf_nodes=None,
                               # минимальное уменьшение загрязнения 
                               min_impurity_decrease=0.001, 
                               # Устарело
                               min_impurity_split=None,
                               # Дерево обучается по части наблюдений (По какой???)
                               bootstrap=True, 
                               # минимальное число наблюдений в узле потомке
                               min_samples_leaf=10, 
                               # минимальное число наблюдений в узле родителе
                               min_samples_split=15,
                               # параметр используется редко, самостоятельно
                               min_weight_fraction_leaf=0.0, 
                               # объем промежуточных результатов в ходе обучения
                               verbose=1, 
                               # число используемых процессоров
                               # n_jobs=None,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения
                               class_weight=None)


grid_search_res = GridSearchCV(model2, param_grid=param_grid, cv=2, 
                               #  выбор критерия качества
                               scoring='accuracy', 
                               #  поведение в случае ошибки при вычислении критерия качества
                               error_score='raise', 
                               #  обучить модель с лучшими значениями параметроы
                               refit=True, 
                               pre_dispatch=None, verbose=0)


In [23]:
grid_search_res.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

GridSearchCV(cv=2, error_score='raise',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=5,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.001,
                                              min_impurity_split=None,
                                              min_samples_leaf=10,
                                              min_samples_split=15,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=80, n_jobs=None,
                                              oob_score=True, random_state=42,
                                              verbose=1, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [4, 

In [24]:
#   Лучший набор значений гиперпараметров

best_hyperparams = grid_search_res.best_params_
print('Лучшие значения гиперпараметров:\n', best_hyperparams)

Лучшие значения гиперпараметров:
 {'max_depth': 6, 'n_estimators': 200}


In [25]:
# Значение критерия качества для лучших значений гиперпараметров 
# Mean cross-validated score of the best_estimator
best_CV_score = grid_search_res.best_score_
print('Лучшее accuracy', best_CV_score)

Лучшее accuracy 0.5540993599512344


In [27]:
# Лучшая модель
best_model = grid_search_res.best_estimator_

# accuracy на тестовом множестве для лучшей модели
test_acc = best_model.score(X_test,y_test)

# Значение критерия качества для лучших значений гиперпараметров 
print('Лучшее accuracy на тестовом множестве', test_acc)

Лучшее accuracy на тестовом множестве 0.5615337043908473


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished


In [28]:
grid_search_res.cv_results_

{'mean_fit_time': array([0.08551633, 0.19652259, 0.42649472, 0.10047781, 0.22599638,
        0.50601649, 0.06397665, 0.21701527, 0.41297543]),
 'std_fit_time': array([0.02448189, 0.00051606, 0.01249373, 0.02252114, 0.00800121,
        0.10598016, 0.00301921, 0.00201297, 0.00202167]),
 'mean_score_time': array([0.01498246, 0.02797902, 0.05852032, 0.00950086, 0.02350104,
        0.05597794, 0.01050091, 0.02945971, 0.04704547]),
 'std_score_time': array([0.00501704, 0.00401628, 0.00847554, 0.00050199, 0.00049984,
        0.01201379, 0.0005002 , 0.00446343, 0.0010035 ]),
 'param_max_depth': masked_array(data=[4, 4, 4, 5, 5, 5, 6, 6, 6],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[30, 100, 200, 30, 100, 200, 30, 100, 200],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_v

In [29]:
means = grid_search_res.cv_results_['mean_test_score']
stds  = grid_search_res.cv_results_['std_test_score']

In [30]:
for mean, std, params in zip(means, stds, grid_search_res.cv_results_['params']):
    print("%0.4f (+/-%0.03f) for %r"  % (mean, std * 2, params))
print()

0.5459 (+/-0.014) for {'max_depth': 4, 'n_estimators': 30}
0.5410 (+/-0.014) for {'max_depth': 4, 'n_estimators': 100}
0.5425 (+/-0.011) for {'max_depth': 4, 'n_estimators': 200}
0.5474 (+/-0.019) for {'max_depth': 5, 'n_estimators': 30}
0.5489 (+/-0.010) for {'max_depth': 5, 'n_estimators': 100}
0.5477 (+/-0.004) for {'max_depth': 5, 'n_estimators': 200}
0.5511 (+/-0.005) for {'max_depth': 6, 'n_estimators': 30}
0.5538 (+/-0.008) for {'max_depth': 6, 'n_estimators': 100}
0.5541 (+/-0.010) for {'max_depth': 6, 'n_estimators': 200}



In [51]:
# Строим предсказание модели
y_pred_train = grid_search_res.predict(X_train)
y_pred_test = grid_search_res.predict(X_test)

In [52]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,3,4,5,6,7,8,9
3,0,0,5,5,0,0,0
4,0,2,31,20,0,0,0
5,0,0,257,210,3,0,0
6,0,0,130,590,38,0,0
7,0,0,8,193,84,0,0
8,0,0,0,23,17,0,0
9,0,0,0,0,1,0,0


In [53]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,3,4,5,6,7,8,9
3,0,0,3,7,0,0,0
4,0,15,63,30,2,0,0
5,0,0,667,313,7,0,0
6,0,0,211,1177,52,0,0
7,0,0,14,344,237,0,0
8,0,0,2,74,50,9,0
9,0,0,0,2,2,0,0


In [54]:
# Строим предсказание модели
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
print(y_pred_train2)
print(y_pred_test2)

[[3.50359878e-03 5.94291055e-03 9.75592564e-02 ... 3.42528349e-01
  6.67058902e-02 2.02918211e-03]
 [4.93635292e-03 1.58467327e-02 1.85708214e-01 ... 2.34394372e-01
  4.51061575e-02 1.29876238e-03]
 [2.63486854e-03 2.47894935e-02 5.70155802e-01 ... 4.47302309e-02
  1.14530953e-02 7.46362181e-05]
 ...
 [2.00729532e-03 1.78222823e-01 1.23199954e-01 ... 1.96631842e-01
  5.07353662e-02 1.74568239e-03]
 [3.16351135e-03 8.08964226e-03 1.11241886e-01 ... 1.97953667e-01
  4.76004615e-02 3.57088819e-04]
 [3.86578432e-03 1.86265548e-02 1.85503351e-01 ... 1.50812029e-01
  3.92673845e-02 2.99955199e-04]]
[[2.72196257e-03 1.21821334e-02 1.40241962e-01 ... 2.83299678e-01
  5.48280368e-02 8.41105265e-04]
 [1.19030523e-03 2.75075384e-02 4.02868035e-01 ... 1.56184066e-01
  1.69865717e-02 4.84759642e-04]
 [2.28298123e-03 2.02593628e-02 1.83687809e-01 ... 1.61875518e-01
  2.34466227e-02 5.68600380e-04]
 ...
 [3.54278944e-03 1.98177075e-02 2.10596740e-01 ... 1.20067040e-01
  1.81687832e-02 7.64985375e-04]

In [55]:
y_pred_train2 = grid_search_res.predict(X_train)
y_pred_test2 = grid_search_res.predict(X_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         0
           4       0.04      1.00      0.07         2
           5       0.55      0.60      0.57       431
           6       0.78      0.57      0.66      1041
           7       0.29      0.59      0.39       143
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

   micro avg       0.58      0.58      0.58      1617
   macro avg       0.24      0.39      0.24      1617
weighted avg       0.67      0.58      0.61      1617



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Можно заметить, что с GridSearch f1-score стало больше, хоть и незначительно

## Калибровка модели

In [32]:
from sklearn.calibration import CalibratedClassifierCV

In [33]:
model_sigmoid = CalibratedClassifierCV(best_model, cv='prefit', method='sigmoid')

#   model_sigmoid = CalibratedClassifierCV(best_model, cv='prefit', method='isotonic')



#   model_sigmoid = CalibratedClassifierCV(grid_search_res, cv=2, method='sigmoid')



In [34]:
#   Небрежность!
#   Для калибровки лучше использовать специальную подвыборку!
#   Есть риск переобучения

model_sigmoid.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished


CalibratedClassifierCV(base_estimator=RandomForestClassifier(bootstrap=True,
                                                             class_weight=None,
                                                             criterion='gini',
                                                             max_depth=6,
                                                             max_features='auto',
                                                             max_leaf_nodes=None,
                                                             min_impurity_decrease=0.001,
                                                             min_impurity_split=None,
                                                             min_samples_leaf=10,
                                                             min_samples_split=15,
                                                             min_weight_fraction_leaf=0.0,
                                                             n_estimators=200,
               

In [35]:
#   Необязательно!
#   Калибровка проводится вовсе НЕ для улучшения качества распознавания классов

y_pred_train3 = model_sigmoid.predict(X_train)
y_pred_test3 = model_sigmoid.predict(X_test)
print(metrics.classification_report(y_pred_test3, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished


              precision    recall  f1-score   support

           4       0.07      0.40      0.11        10
           5       0.61      0.60      0.60       480
           6       0.69      0.57      0.62       906
           7       0.40      0.52      0.45       217
           8       0.06      1.00      0.12         4

    accuracy                           0.57      1617
   macro avg       0.37      0.62      0.38      1617
weighted avg       0.62      0.57      0.59      1617



In [38]:
model_sigmoid.predict_proba(X_test)[:5, :]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished


array([[0.01162454, 0.31583622, 0.61372563, 0.05483724, 0.00397637],
       [0.02736551, 0.16027709, 0.63116541, 0.16089995, 0.02029205],
       [0.00713924, 0.13543534, 0.73734125, 0.11264779, 0.00743639],
       [0.00491791, 0.04184414, 0.66343427, 0.27441608, 0.01538759],
       [0.19081482, 0.30462427, 0.34511261, 0.12373371, 0.0357146 ]])